In [1]:
import pandas as pd
import numpy as np
import gc

In [2]:
dtypes = {"review_id": str, "review_text": str, "n_votes": np.int8, "n_comments" : np.int8, "rating":np.int8}

In [3]:
train_data = pd.read_csv("../input/goodreads-books-reviews-290312/goodreads_train.csv", usecols=["review_id", "review_text", "n_votes", "n_comments", "rating"],dtype=dtypes)
gc.collect()

46

In [4]:
del(dtypes)
gc.collect()

23

In [5]:
train_pos = train_data[(train_data["n_votes"] >= 0) & (train_data["n_comments"] >= 0)]
gc.collect()

23

In [6]:
del(train_data)
gc.collect()

23

In [7]:
from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=2, random_state=42, test_size=0.85)
gc.collect()

0

In [8]:
for train_index, _ in sss.split(train_pos.drop("rating", axis=1), train_pos["rating"]):
  df_train = train_pos.iloc[train_index]

In [9]:
del(train_index)
del(_)
del(train_pos)
gc.collect()

46

In [4]:
from fastai.text.all import *

ModuleNotFoundError: No module named 'fastai'

In [11]:
dls = TextDataLoaders.from_df(df=df_train, is_lm=True, text_col="review_text", label_col="rating")
dls.show_batch(max_n=3)
gc.collect()

,text,text_
0,"xxbos xxmaj this review was originally posted on ( un)conventional xxmaj bookviews \n▁ xxmaj the xxmaj duke 's xxmaj accidental xxmaj wife had a lot of sweetness inside , even if there was quite a bit of sadness as well , because the protagonists did n't really understand each other at first . \n▁ xxmaj the xxmaj dukes of xxmaj war series is a really good one , and i loved getting","xxmaj this review was originally posted on ( un)conventional xxmaj bookviews \n▁ xxmaj the xxmaj duke 's xxmaj accidental xxmaj wife had a lot of sweetness inside , even if there was quite a bit of sadness as well , because the protagonists did n't really understand each other at first . \n▁ xxmaj the xxmaj dukes of xxmaj war series is a really good one , and i loved getting to"
1,"i liked xxmaj georgia and xxmaj katya 's grandparents , but i wished that , again , xxmaj georgia had been stronger . \n▁ i hope that this series gets better as it goes along , but it 's still iffy if i read the other books . xxmaj this book did n't really pop out at me . xxbos xxmaj this graphic novel made me giggle quietly to myself as i","liked xxmaj georgia and xxmaj katya 's grandparents , but i wished that , again , xxmaj georgia had been stronger . \n▁ i hope that this series gets better as it goes along , but it 's still iffy if i read the other books . xxmaj this book did n't really pop out at me . xxbos xxmaj this graphic novel made me giggle quietly to myself as i read"
2,"immense personal bias in its favor , which undoubtedly colors my view . xxmaj when i read it , i immediately saw the parallel between the sisters in the book and my sister and xxup i , and between her daughters as well . i gave it to her to read without saying anything , and she saw it as well . xxbos * * spoiler alert * * \n▁ xxmaj lisa","personal bias in its favor , which undoubtedly colors my view . xxmaj when i read it , i immediately saw the parallel between the sisters in the book and my sister and xxup i , and between her daughters as well . i gave it to her to read without saying anything , and she saw it as well . xxbos * * spoiler alert * * \n▁ xxmaj lisa xxmaj"


60

In [12]:
learn = language_model_learner(dls,  AWD_LSTM, metrics=[F1Score(average="macro"),Perplexity()],  wd=0.1).to_fp16()
gc.collect()

50

In [13]:
learn.fit_one_cycle(5)
gc.collect()

epoch,train_loss,valid_loss,f1_score,perplexity,time
0,4.176833,3.892157,0.011327,49.016518,59:07
1,4.128623,3.847522,0.012442,46.876781,1:00:37
2,4.090974,3.838825,0.013486,46.470837,1:01:02
3,4.087087,3.834248,0.013960,46.258610,1:01:54
4,4.080578,3.832944,0.014057,46.198341,1:03:00


0

In [14]:
learn.save_encoder('finetuned')
gc.collect()

23

In [15]:
dls_cls = TextDataLoaders.from_df(df=df_train, text_col="review_text", label_col="rating")
dls_cls.show_batch(max_n=3)
gc.collect()

,text,category
0,"xxbos xxmaj this review was posted on our blog xxmaj the xxmaj three xxmaj bookateers \n▁ i have been meaning to write this review for a very very long time , however i have been putting it off as i did n't know how to arrange my thoughts into a comprehensive , cohesive sentence . xxmaj thinking about xxmaj everything xxmaj changes makes my my brain go xxup xxunk because there is just so much to discuss . \n▁ xxmaj so , what was is it about this book that turns my brain to mush ? xxmaj its not just one thing , its a whole lot of things combined . xxmaj firstly it was such a different and refreshing read , the story line was full of many new challenges where things happened that made me go wtf . xxmaj secondly , the characters experienced difficulties that i have",4
1,"xxbos xxmaj grant xxmaj morrison re - re - invigorated my love of superhero comics in the late 80s / early 90s , building on the momentum forged by xxmaj alan xxmaj moore and others . xxmaj the minute xxmaj buddy xxmaj baker looked out of an issue xxup animal xxup man at me , sitting in some shitty single room flat at xxmaj coventry xxmaj xxunk in xxmaj england , and really * saw * me , i knew i was going to be a fan for life ( well that , and realizing that i was on xxmaj mr . xxmaj nobody 's side in his bid for the xxmaj american xxmaj presidency as it ran in xxup doom xxup patrol ) . \n▁ xxmaj so i got to watch with pride and joy as xxmaj morrison grew in popularity , bringing his own peculiar take on modern",5
2,"xxbos * * spoiler alert * * \n▁ xxmaj is it just me , or is anyone else really starting to like xxmaj haymitch ? \n▁ xxmaj bet you did n't see this coming . xxmaj moi , xxmaj kira , reading xxmaj catching xxmaj fire and giving it four stars ! \n▁ xxmaj the xxmaj hunger xxmaj games earned an uneasy 2.5 stars from me . xxmaj the most annoying part ? i was n't 100 % sure why . xxmaj katniss , probably . xxmaj the almost absurd seriousness of her voice and her behavior toward xxmaj peeta tipped me ( though i did have to give her kudos for being absolutely kick - ass ) . xxmaj the cavalier treatment of xxmaj haymitch 's alcoholism , the total lack of sensical world - building and the sentence fragments grated on me . i enjoyed it , yes",4


100

In [16]:
learn = text_classifier_learner(dls_cls, AWD_LSTM, drop_mult=0.5, metrics=F1Score(average="macro"))
gc.collect()

32595

In [17]:
learn = learn.load_encoder('finetuned')
gc.collect()

33

In [23]:
# learn.unfreeze()
learn.fit_one_cycle(5, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,f1_score,time
0,0.832682,0.932392,0.531965,12:46
1,0.874576,0.954068,0.513998,12:46
2,0.809269,0.948113,0.539850,12:48
3,0.766068,0.955827,0.540215,12:43


KeyboardInterrupt: 

In [24]:
learn.show_results()

,text,category,category_
0,"xxbos xxmaj in almost every case , xxmaj indian societies have been revealed to be older , grander , and more complex than was thought possible even twenty years ago . … xxmaj cultures are like books , the anthropologist xxmaj claude levi - strauss once remarked , each a volume in the great library of humankind . xxmaj in the sixteenth century , more books were burned than ever before or since . xxmaj how many xxmaj xxunk vanished ? xxmaj how many xxmaj xxunk ? xxmaj what great works of painting , sculpture , architecture , and music vanished or never were created ? xxmaj languages , prayers , dreams , habits , and hopes -- all gone . xxmaj and not just once , but over and over again . xxmaj in our antibiotic era , how can we imagine what it means to have entire ways",5,4
1,"xxbos xxmaj fans of xxmaj mitch xxmaj davies had high hopes for this one , but i think it 's fair to say that reactions have been mixed . xxmaj of course , with the critical success of xxmaj mapping xxmaj xxunk and xxmaj davies ' reputation for inventive structuring , this long - awaited release was meant to solidify his status as a literary virtuoso . xxmaj my own feeling is he fell short of the mark , though with strong work that at times brought him close . xxmaj as i was tallying my pluses and minuses , i made the mistake of looking at the many excellent xxmaj goodreads reviews already out there and now realize that most of my thoughts have been covered by them . i should have just left it at that - end of review . xxmaj but instead i probed deep into",3,4
2,"xxbos xxmaj oh my goodness . i do n't really think i can capture this book in a review , it 's s xxrep 3 o intense . xxmaj it 's kind of untamed and wild and messy and beautiful . xxmaj the plot and explanations of the world could use a tiny bit of work , but the writing and the characters and the ideas make up for it in spades . xxmaj sure it 's not "" typical "" xxup ya , but come on , it 's beautiful . xxmaj if you ca n't handle a little sex then pass it by but it 's about life , sex is a part of life , it 's totally warranted in my mind . xxmaj it 's a lot like xxmaj looking for xxmaj alaska just in intensity and themes and stuff , but this is n't humorous",5,4
3,"xxbos i know that not all children turn out to be like their parents , but man , that xxmaj keirran is something else . xxmaj it 's the equivalent of xxmaj bill and xxmaj hillary xxmaj clinton having xxmaj justin xxmaj bieber as a son . \n▁ xxmaj judge me how you will , but not since xxmaj harry xxmaj potter # 5 have my feelings been so heightened by the events of a book . i want to punch someone ( keirran ) . i want to yell at him . i want to pick him up and shake some goddamn sense into him . i want to hold xxmaj annwyl 's hand and stay with her into that sad goodnight . i want to shake some sense into xxmaj kenzie . i want to pull xxmaj ethan in for a hug and never let him go .",5,4
4,"xxbos xxmaj ugh , the meh books are the worst . xxmaj fuck it , xxup xxunk xxup time ! \n▁ xxmaj cyborg xxmaj cinderella is xxmaj cool \n▁ i found the ideas and aesthetic behind xxmaj cinder very appealing . xxmaj it 's difficult to resist the allure of a female cyborg mechanic protagonist , and the cyberpunk look is never a hard sell for me . xxmaj even when the story and i were not getting along , the cyborg bits ( very xxmaj ghost in the xxmaj shell , btw ) would pop up to remind me that this was a really neat concept . xxmaj on - xxunk computer displays , hollow compartments in limbs , the whole cyber - foot - as - glass slipper bit , that was all pretty clever and fun to visualize . xxmaj likewise , xxmaj moon xxmaj people ,",2,2
5,"xxbos xxmaj this is a fantastic anthology . xxmaj every author added a story that i enjoyed . xxmaj while expressed in individual ways and differing stories , there are elements of tragedy and a very human horror that run throughout the anthology as a whole . xxmaj the agony of being deni

In [26]:
learn.export("model2.pkl")

In [2]:
dtypes = {"review_id": str, "review_text": str}

In [3]:
data = pd.read_csv("data/test/goodreads_test.csv",dtype=dtypes)
gc.collect()

160